# Port Expenses API Example

## Run this example in [Colab](https://colab.research.google.com/github/SignalOceanSdk/SignalSDK/blob/master/docs/examples/jupyter/PortExpensesAPI/PortExpensesAPI.ipynb) 

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [ ]:
pip install signal-ocean

In [5]:
signal_ocean_api_key = '' #replace with your subscription key

## Port expenses API
The Port Expenses API retrieves expenses breakdown for a given port. First create connection towards
Port Expenses API in order to find available ports by name:

In [6]:
from signal_ocean import Connection, PortExpensesAPI
from signal_ocean.port_expenses import PortFilter
from datetime import datetime

connection = Connection(api_key=signal_ocean_api_key)
pe_api = PortExpensesAPI(connection)

#### Get the port expenses for an IMO
Now retrieve the expenses for the port of Ningbo for a given vessel IMO:

In [7]:
ningbo = pe_api.get_ports(PortFilter(name_like='Ningbo'))[0]
pe_api.get_port_expenses(imo = 9867621, port_id = ningbo.id)

PortExpenses(port_id=3302, port_canal=0, towage=13381, berth=4257, port_dues=26109, lighthouse=0, mooring=0, pilotage=10569, quay=0, anchorage=851, agency_fees=16716, other=4809, suez_dues=0, total_cost=76692, miscellaneous_dues=0, is_estimated=False, canal_dues=0, berth_dues=4257, lighthouse_dues=0, mooring_unmooring=0, quay_dues=0, anchorage_dues=851, port_agents=[])

The result contains a breakdown of the costs as well as the total cost.
In cases where we do not have the exact port costs modelled in our system, a total cost based on historical actual DAs is returned. In that case the is_estimated is True.

#### Optional port expenses calculation parameters
Depending on the port, the expenses calculation can be more precise by passing some extra parameters.
Retrieve the list of the parameters for the port of Ningbo.

In [8]:
pe_api.get_required_formula_parameters(port_id = ningbo.id, vessel_type_id = 1)

[]

Get again the expenses for the port of Ningbo, this time by also passing one of the optional
parameters retrieved above i.e. estimated time of berth:

In [9]:
pe_api.get_port_expenses(imo = 9867621, port_id = ningbo.id, estimated_time_of_berth=datetime(2019, 2, 27, 17, 48, 11))

PortExpenses(port_id=3302, port_canal=0, towage=13381, berth=4257, port_dues=26109, lighthouse=0, mooring=0, pilotage=8627, quay=0, anchorage=851, agency_fees=16716, other=4809, suez_dues=0, total_cost=74751, miscellaneous_dues=0, is_estimated=False, canal_dues=0, berth_dues=4257, lighthouse_dues=0, mooring_unmooring=0, quay_dues=0, anchorage_dues=851, port_agents=[])

#### Get the port expenses for a model vessel
Port expenses can also be calculated based on a model vessel. Retrieve all
available vessel types:

In [10]:
vessel_types = pe_api.get_vessel_types()
print(vessel_types)

(VesselType(id=1, name='Tanker'), VesselType(id=3, name='Dry'), VesselType(id=4, name='Container'), VesselType(id=5, name='LNG'), VesselType(id=6, name='LPG'))


Get expenses for Ningbo for dry vessels as of 8-3-2020:

In [11]:
dry_vessel_type = next(vt for vt in vessel_types if vt.name=='Dry')
pe_api.get_port_model_vessel_expenses(port_id = ningbo.id, vessel_type_id = dry_vessel_type.id, formula_calculation_date = datetime(2020, 3, 8, 0, 0, 0))

PortExpenses(port_id=3302, port_canal=0, towage=0, berth=1070, port_dues=19687, lighthouse=0, mooring=0, pilotage=8067, quay=0, anchorage=214, agency_fees=4411, other=1117, suez_dues=0, total_cost=34566, miscellaneous_dues=0, is_estimated=False, canal_dues=0, berth_dues=1070, lighthouse_dues=0, mooring_unmooring=0, quay_dues=0, anchorage_dues=214, port_agents=[])